In [ ]:
from scipy import stats

In [ ]:
## ici flag_resiliation est la variable cible ayant les valeurs 0 ou 1.
## la variable nb_reengagements est quantitative.
## ce test va permettre de mesurer l'écart entre des distributions de la variable quantitative au sein des deux groupes
## 0, et 1. Et plus la statistique de test est grande plus l'écart entre les distribution est grande et plus 
## la variable quantitative est associé à la variable qualitative.

stats.kstest(df.loc[df["flag_resiliation"]=="0","nb_reengagements"],df.loc[df["flag_resiliation"]=="1","nb_reengagements"])

In [ ]:
def rapport_corr(x,y):
    '''
    x est une variable qualitative
    y est une variable quantitative
    elle donne le pourcentage de variabilité de y capté par la difference des modalités de x
    il va donc mesurer l'intensité de la liaison entre x et y.
    '''
    somme = 0
    for facteur in x.value_counts().index:
        ind = x[x == facteur].dropna(axis=0).index
        classe = y.iloc[ind,]
        somme = somme + len(classe)*np.sum((np.mean(classe)-np.mean(y))**2)
        y_ecart = np.sum((y-np.mean(y))**2)
        rc = somme/y_ecart
    return rc

rapport_corr(df.flag_resiliation,df.nb_reengagements)

In [1]:
## Chragement des librairie à utiliser

from tqdm import tqdm  
from dateutil.relativedelta import relativedelta
from dask import dataframe as df_dask
import datetime
import pandas as pd, csv, numpy as np, os, re, warnings, seaborn as sns, matplotlib.pyplot as plt, plotly.express as px
from scipy import stats
#%matplotlib notebook
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option("display.max_columns",None)

In [2]:
## import des fichiers

df_inter_camp = pd.read_pickle("P:\SIEGE\DEVELOPPEMENT_RESEAU\Marketing Opérationnel\MKO_nouvelle_orga\9 - Data - Market Opé\Autres travaux data\Etude_Pression_Com_Rel\Output_Files\INTER_CAMP_LINK_R_EMAIL")

df_inter_SMC = pd.read_pickle("P:\SIEGE\DEVELOPPEMENT_RESEAU\Marketing Opérationnel\MKO_nouvelle_orga\9 - Data - Market Opé\Autres travaux data\Etude_Pression_Com_Rel\Output_Files\INTER_SMC_EMAIL")

df_test_anc =  pd.read_pickle(r"P:\SIEGE\DEVELOPPEMENT_RESEAU\Marketing Opérationnel\MKO_nouvelle_orga\9 - Data - Market Opé\Autres travaux data\Etude_Pression_Com_Rel\Output_Files\NOIDE_ANC")

In [5]:
## visualisation de "df_test_anc" utile pour filtrer sur les adhérents ayant au plus 3 ans d'ancienneté

df_test_anc

,NOIDE,Anciennete
0,0100000001,0.0
78,0100009577,3.0
121,0100011308,1.0
150,0100012161,3.0
152,0100012278,1.0
...,...,...
1749453,0108968816,2.0
1749454,0108968817,2.0
1749455,0108968818,1.0
1749456,0108968821,1.0


In [6]:
## Création d'une colonne clé afin d'effectuer des jointures

df_inter_SMC["Cle"] = df_inter_SMC["NOIDE"] + df_inter_SMC["LIBCPGSIM"]

df_inter_camp["Cle"] = df_inter_camp["NOIDE"] + df_inter_camp["LIBCPGSIM"]

In [7]:
## Jointure des fichiers

df_total = df_inter_camp.merge(df_inter_SMC, on="Cle", how="left")

df_total = df_total.merge(df_test_anc, left_on="NOIDE_x", right_on="NOIDE")

In [9]:
## Libération de la mémoire

del df_inter_camp

del df_inter_SMC

gc.collect()

20

In [10]:
## Suppression des colonnes redondantes

col = [col for col in df_total.columns if col.endswith("_y")]

df_total.drop(col,axis=1,inplace=True)

In [11]:
## Suppression des colonnes

df_total.drop(["OUVERT_36","CLIC_36","OBJETCAMP","CODCBLCPG","Cle"],axis=1,inplace=True)

In [12]:
## récupération des lignes renseignées sur les colonnes "OUVERT_36_MOD" et "CLIC_36_MOD"

df_total_notNa = df_total[~df_total["OUVERT_36_MOD"].isin([np.nan,"Non_recu"]) & ~df_total["CLIC_36_MOD"].isin([np.nan,"Non_recu"])].copy()

In [13]:
## libération de la mémoire

del df_total

gc.collect()

0

In [14]:
## Ordonne le dataframe par ordre croissant de date d'interaction de campagne et de NOIDE

df_total_notNa = df_total_notNa.sort_values(["NOIDE_x","DATE_INTER"])

df_total_notNa.head()

,NOIDE_x,CODCPGSIM_x,LIBCPGSIM_x,DATE_INTER,CAMPCHAPEAU_x,INTER_CANAL,LIBCPGSMC,CODCNL,STATUT_ENVOI,NBRE_OUVERTURE_36,NBRE_CLIC_36,OUVERT_36_MOD,CLIC_36_MOD,NBRE_OUVERTURE_36_MOD,STATUT_ENVOI_MOD,NOIDE,Anciennete
438329,0100000001,5663,NL - 202009 - MGEN AVANTAGE,2020-09-18,NL - 202009 - MGEN AVANTAGE,INTER_MAIL,NL - 202009 - MGEN AVANTAGE - MP 45 ans et plus,EMAIL,email_delivre,NaN,NaN,0,0,NaN,email_delivre,0100000001,0.0
438328,0100000001,6540,NL - 202101 - MSH - E,2021-01-28,NL - 202101 - MSH - E,INTER_MAIL,NL - 202001 - MSH Vague,EMAIL,email_delivre,NaN,NaN,0,0,NaN,email_delivre,0100000001,0.0
751942,0100009577,4678,NL - 202002 - Mgen Services habitat,2020-02-26,NL - 202002 - Mgen Services habitat,INTER_MAIL,NL - 202002 - MSH,EMAIL,email_delivre,1.0,NaN,1,0,1.0,email_delivre,0100009577,3.0
751944,0100009577,5663,NL - 202009 - MGEN AVANTAGE,2020-09-22,NL - 202009 - MGEN AVANTAGE,INTER_MAIL,NL - 202009 - MGEN AVANTAGE - MP 45 ans et plus,EMAIL,email_delivre,NaN,NaN,0,0,NaN,email_delivre,0100009577,3.0
751945,0100009577,6540,NL - 202101 - MSH - E,2021-02-01,NL - 202101 - MSH - E,INTER_MAIL,NL - 202001 - MSH Vague,EMAIL,email_delivre,NaN,NaN,0,0,NaN,email_delivre,0100009577,3.0


In [15]:
## Groupage au sein de chaque groupe de NOIDE identique et ajout du ranking de l'email

df_total_notNa["RANK"] = df_total_notNa.groupby("NOIDE_x")["DATE_INTER"].rank(method="first", ascending=True)

In [16]:
## Renommage du ranking

df_total_notNa["Email"] = "email_" + (df_total_notNa["RANK"].astype(int)).astype(str)

df_total_notNa.head()

,NOIDE_x,CODCPGSIM_x,LIBCPGSIM_x,DATE_INTER,CAMPCHAPEAU_x,INTER_CANAL,LIBCPGSMC,CODCNL,STATUT_ENVOI,NBRE_OUVERTURE_36,NBRE_CLIC_36,OUVERT_36_MOD,CLIC_36_MOD,NBRE_OUVERTURE_36_MOD,STATUT_ENVOI_MOD,NOIDE,Anciennete,RANK,Email
438329,0100000001,5663,NL - 202009 - MGEN AVANTAGE,2020-09-18,NL - 202009 - MGEN AVANTAGE,INTER_MAIL,NL - 202009 - MGEN AVANTAGE - MP 45 ans et plus,EMAIL,email_delivre,NaN,NaN,0,0,NaN,email_delivre,0100000001,0.0,1.0,email_1
438328,0100000001,6540,NL - 202101 - MSH - E,2021-01-28,NL - 202101 - MSH - E,INTER_MAIL,NL - 202001 - MSH Vague,EMAIL,email_delivre,NaN,NaN,0,0,NaN,email_delivre,0100000001,0.0,2.0,email_2
751942,0100009577,4678,NL - 202002 - Mgen Services habitat,2020-02-26,NL - 202002 - Mgen Services habitat,INTER_MAIL,NL - 202002 - MSH,EMAIL,email_delivre,1.0,NaN,1,0,1.0,email_delivre,0100009577,3.0,1.0,email_1
751944,0100009577,5663,NL - 202009 - MGEN AVANTAGE,2020-09-22,NL - 202009 - MGEN AVANTAGE,INTER_MAIL,NL - 202009 - MGEN AVANTAGE - MP 45 ans et plus,EMAIL,email_delivre,NaN,NaN,0,0,NaN,email_delivre,0100009577,3.0,2.0,email_2
751945,0100009577,6540,NL - 202101 - MSH - E,2021-02-01,NL - 202101 - MSH - E,INTER_MAIL,NL - 202001 - MSH Vague,EMAIL,email_delivre,NaN,NaN,0,0,NaN,email_delivre,0100009577,3.0,3.0,email_3


In [18]:
## taille du fichier

df_total_notNa.shape

(1460518, 19)

In [25]:
## Différentes valeurs de "CLIC_36_MOD"

df_total_notNa["CLIC_36_MOD"].value_counts(dropna=False)

0    1431730
1      28788
Name: CLIC_36_MOD, dtype: int64

In [20]:
## changement du type des colonnes 

df_total_notNa["OUVERT_36_MOD"] = df_total_notNa["OUVERT_36_MOD"].astype(int)

df_total_notNa["CLIC_36_MOD"] = df_total_notNa["CLIC_36_MOD"].astype(int)

In [85]:
## calcul du nombre d'ouverture et de clic et d'ouvreur par rang de l'email

df_sorti_notna = df_total_notNa.groupby("Email").agg({"OUVERT_36_MOD":["sum","count"],"CLIC_36_MOD":["sum","count"]})

df_sorti_notna.columns = [x+"_"+y for x,y in df_sorti_notna.columns]

In [62]:
df_total_notNa.head()

,NOIDE_x,CODCPGSIM_x,LIBCPGSIM_x,DATE_INTER,CAMPCHAPEAU_x,INTER_CANAL,LIBCPGSMC,CODCNL,STATUT_ENVOI,NBRE_OUVERTURE_36,NBRE_CLIC_36,OUVERT_36_MOD,CLIC_36_MOD,NBRE_OUVERTURE_36_MOD,STATUT_ENVOI_MOD,NOIDE,Anciennete,RANK,Email
438329,0100000001,5663,NL - 202009 - MGEN AVANTAGE,2020-09-18,NL - 202009 - MGEN AVANTAGE,INTER_MAIL,NL - 202009 - MGEN AVANTAGE - MP 45 ans et plus,EMAIL,email_delivre,NaN,NaN,0,0,NaN,email_delivre,0100000001,0.0,1.0,email_1
438328,0100000001,6540,NL - 202101 - MSH - E,2021-01-28,NL - 202101 - MSH - E,INTER_MAIL,NL - 202001 - MSH Vague,EMAIL,email_delivre,NaN,NaN,0,0,NaN,email_delivre,0100000001,0.0,2.0,email_2
447198,0100013571,4678,NL - 202002 - Mgen Services habitat,2020-02-26,NL - 202002 - Mgen Services habitat,INTER_MAIL,NL - 202002 - MSH,EMAIL,email_delivre,NaN,NaN,0,0,NaN,email_delivre,0100013571,3.0,1.0,email_1
447199,0100013571,5663,NL - 202009 - MGEN AVANTAGE,2020-09-24,NL - 202009 - MGEN AVANTAGE,INTER_MAIL,NL - 202009 - MGEN AVANTAGE - MP 45 ans et plus,EMAIL,email_delivre,NaN,NaN,0,0,NaN,email_delivre,0100013571,3.0,2.0,email_2
447201,0100013571,6256,NL - 202012 - Mgen Avantage,2020-12-11,NL - 202012 - Mgen Avantage,INTER_MAIL,NL - 202012 - MGEN AVANTAGE - MSP v1,EMAIL,email_delivre,NaN,NaN,0,0,NaN,email_delivre,0100013571,3.0,3.0,email_3


In [88]:
## renommage des colonnes

df_sorti_notna.drop("OUVERT_36_MOD_count",axis=1,inplace=True)

df_sorti_notna.rename(columns={"OUVERT_36_MOD_sum":"Nbre_Ouvreur","CLIC_36_MOD_sum":"Nbre_cliqueur","CLIC_36_MOD_count":"Nbre_adh"}, inplace=True)

In [89]:
df_sorti_notna.head()

,Nbre_Ouvreur,Nbre_cliqueur,Nbre_adh
Email,,,
email_1,14885,642,38264
email_10,7373,352,22590
email_11,6701,286,20567
email_12,6107,251,18718
email_13,5361,195,16619


In [90]:
## Calcul des proportions

df_sorti_notna["%ouvreur"] = round(df_sorti_notna["Nbre_Ouvreur"]/df_sorti_notna["Nbre_adh"],2)*100

df_sorti_notna["%cliqueur"] = round(df_sorti_notna["Nbre_cliqueur"]/df_sorti_notna["Nbre_adh"],2)*100

df_sorti_notna["%cliqueur/ouvreur"] = round(df_sorti_notna["Nbre_cliqueur"]/df_sorti_notna["Nbre_Ouvreur"],2)*100

df_sorti_notna.sort_values("Nbre_Ouvreur", inplace=True, ascending=False)

df_sorti_notna.reset_index(inplace=True)

In [92]:
## visualisation des 25 premières lignes

df_sorti_notna.head(25)

,Email,Nbre_Ouvreur,Nbre_cliqueur,Nbre_adh,%ouvreur,%cliqueur,%cliqueur/ouvreur
0,email_1,14885,642,38264,39.0,2.0,4.0
1,email_2,13316,626,37227,36.0,2.0,5.0
2,email_3,12625,643,36087,35.0,2.0,5.0
3,email_4,12255,709,34256,36.0,2.0,6.0
4,email_5,11189,653,32314,35.0,2.0,6.0
5,email_6,10428,641,30480,34.0,2.0,6.0
6,email_7,9562,466,28430,34.0,2.0,5.0
7,email_8,8839,392,26533,33.0,1.0,4.0
8,email_9,8157,362,24596,33.0,1.0,4.0
9,email_10,7373,352,22590,33.0,2.0,5.0


In [94]:
## Filtrage sur les 20 premières lignes

df_sorti_notna_test = df_sorti_notna.iloc[0:20,].copy()

In [ ]:
## Représentation de l'histogramme de "Proportion d'ouvreur"

fig = px.histogram(df_sorti_notna_test, x="Email",y="%ouvreur",
                   labels={"Email":"Rang d'email reçu"},
                   category_orders={"Email":["email_1","email_2","email_3",
                "email_4","email_5","email_6","email_7","email_8","email_9","email_10"]},text_auto=True)

fig.update_yaxes(title="Proportion d'ouvreur")

fig.show()

In [ ]:
## Représentation de l'histogramme de "Proportion de cliqueur"

fig = px.histogram(df_sorti_notna_test, x="Email",y="%cliqueur",
                   labels={"Email":"Rang d'email reçu"},
                   category_orders={"Email":["email_1","email_2","email_3",
                "email_4","email_5","email_6","email_7","email_8","email_9","email_10"]},text_auto=True)

fig.update_yaxes(title="Proportion de cliqueur")

fig.show()

In [ ]:
## Représentation de l'histogramme de "Proportion de cliqueur/ouvreur"

fig = px.histogram(df_sorti_notna_test, x="Email",y="%cliqueur/ouvreur", category_orders={"Email":["email_1","email_2","email_3",
                "email_4","email_5","email_6","email_7","email_8","email_9","email_10"]},text_auto=True)

fig.show()